In [17]:
# tutorial: http://stanford.edu/~mgorkove/cgi-bin/rpython_tutorials/Using_Python_to_Extract_Tables_From_PDFs.php
import requests

# get apikey from https://pdftables.com/
apiKey = "3bxr701xf0c1"

def pdfToTable(PDFfilename, apiKey, fileExt, downloadDir):
    fileData = (PDFfilename, open(PDFfilename, 'rb'))
    files = {'f': fileData}
    postUrl = "https://pdftables.com/api?key={0}&format={1}".format(apiKey, fileExt)
    response = requests.post(postUrl, files=files)
    response.raise_for_status()
    with open(downloadDir, "wb") as f:
        f.write(response.content)

In [19]:
# read the pdf and save it in csv
pdfToTable(PDFfilename, apiKey, fileExt = "csv", 
           downloadDir = "career_launch_co.csv")

In [211]:
import pandas as pd
data = pd.read_csv("career_launch_co.csv")
data.head()

,Wealthfront’s Career-,Unnamed: 1
0,Launching Companies List,NaN
1,2017 EDITION,NaN
2,A,NaN
3,Act-On So*ware,Cloud-based integrated markeDng plaEorm
4,"Portland, OR",NaN


In [212]:
# list column names
list(data)

# rename columns
data.columns = ["Company", "Business"]

In [216]:
# remove length == 1 in 'Company' column
data = data[data['Company'].map(
    lambda x: len(x) if isinstance(x, str) else 0) != 1]

In [217]:
# remove first and second rows
data = data.drop(data.index[[0, 1]])

In [219]:
data.head(10)

,Company,Business
0,Act-On So*ware,Cloud-based integrated markeDng plaEorm
1,"Portland, OR",NaN
2,Affirmed Networks,Mobile network soluDons
3,"Boston, MA",NaN
4,Alien Vault,NaN
5,NaN,Threat management plaEorm
6,"San Mateo, CA",NaN
7,Anaplan,Financial planning and modeling soHware
8,"San Francisco, CA",NaN
9,App Annie,Business intelligence soluDons


In [220]:
# drop rows contain "Wealthfront" in the "company" column
data = data[~data["Company"].str.contains("Wealthfront", na=False, regex=True)]

In [221]:
# tidy up columns
# issue1: the company column has both a company name and a location
# issue2: the business column has company descriptions split into two rows sometimes
# issue3: we need a location column. This should be referred from the company col

# strategy: consolidate all info in the columns that have the location info
# keep company info and business info in varibles: company and business
# when iterating each row

import re

data['Location'] = "NA"

company = ""
business = ""
for index, row in data.iterrows():
    
    if re.match(r'.+, \w\w', str(row["Company"])):
        row["Location"] = row["Company"]
        row["Company"] = company
        row["Business"] = business
        
        # reset
        business = ""
        company = ""

    elif isinstance(row["Company"], str):
        company = row["Company"]
        
        if isinstance(row["Business"], str):
            business = row["Business"] + business
    
            
    else: # NaN column
        if isinstance(row["Business"], str):
            business = row["Business"] + business     

In [222]:
data.head(20)

,Company,Business,Location
0,Act-On So*ware,Cloud-based integrated markeDng plaEorm,NA
1,Act-On So*ware,Cloud-based integrated markeDng plaEorm,"Portland, OR"
2,Affirmed Networks,Mobile network soluDons,NA
3,Affirmed Networks,Mobile network soluDons,"Boston, MA"
4,Alien Vault,NaN,NA
5,NaN,Threat management plaEorm,NA
6,Alien Vault,Threat management plaEorm,"San Mateo, CA"
7,Anaplan,Financial planning and modeling soHware,NA
8,Anaplan,Financial planning and modeling soHware,"San Francisco, CA"
9,App Annie,Business intelligence soluDons,NA


In [223]:
data = data[data["Location"] != "NA"]

In [227]:
# reset index
data = data.reset_index(drop=True)
data

,Company,Business,Location
0,Act-On So*ware,Cloud-based integrated markeDng plaEorm,"Portland, OR"
1,Affirmed Networks,Mobile network soluDons,"Boston, MA"
2,Alien Vault,Threat management plaEorm,"San Mateo, CA"
3,Anaplan,Financial planning and modeling soHware,"San Francisco, CA"
4,App Annie,Business intelligence soluDons,"San Francisco, CA"
5,AppDynamics,ApplicaDon performance management soluDons for,"San Francisco, CA"
6,Automa<c,Content management plaEorm for blogs and websites,"San Francisco, CA"
7,Avalara,Sales tax compliance soluDons,"SeaAle, WA"
8,Avvo,"Legal marketplace, directory, and quesDon-","SeaAle, WA"
9,Big Switch Networks,SDN white box switching,"Santa Clara, CA"


In [228]:
data.to_csv("cleand_career_launch_co.csv", encoding='utf-8')